Van Gysel et al also prune to top 2^16 most frequent words which also allows them to fit in uint16

In [40]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
import itertools as it
from functools import partial
import nltk
from string import punctuation
import pickle
from collections import Counter

PATH_DATA = '../data/amazon/food/reviews_df.msg'
stop = nltk.corpus.stopwords.words('english')

In [2]:
stemmer = nltk.stem.SnowballStemmer('english')

In [3]:
df = pd.read_msgpack(PATH_DATA)#.head(10000)

In [4]:
%%time
purge_words = stop + list(punctuation) + ['"', '\'', '\\']
data_words = df.Text\
    .apply(lambda row: [stemmer.stem(word.lower())
                        for word in nltk.word_tokenize(row) 
                        if word.lower() not in purge_words])

CPU times: user 12min 52s, sys: 888 ms, total: 12min 52s
Wall time: 12min 53s


In [41]:
%%time
# Find most common words
c = Counter()
_ = data_words.map(lambda words: c.update(words))
thresh = 2**15-1  # should be 2**16 but I want less
freq_words, _ = zip(*c.most_common(thresh))
freq_words = set(freq_words)

CPU times: user 4.83 s, sys: 8 ms, total: 4.84 s
Wall time: 4.87 s


In [62]:
%%time
# Go through again and filter out uncommon words
data_words = data_words\
    .map(lambda words: [word for word in words if word in freq_words])

CPU times: user 13min 52s, sys: 548 ms, total: 13min 53s
Wall time: 13min 54s


In [63]:
vocab = set(it.chain(*data_words))
vocab_size = len(vocab) + 1  # +1 for UNK

max_len = max(map(len, data_words))

In [64]:
vocab_size

32768

In [65]:
%%time
le = LabelEncoder()
le.fit(list(vocab))
le.classes_ = np.insert(le.classes_, 0, '')  # UNK tok @ 0

CPU times: user 40 ms, sys: 0 ns, total: 40 ms
Wall time: 40.2 ms


In [66]:
enc_d = dict(zip(le.classes_, range(len(le.classes_))))

In [67]:
%%time
data_words_enc = data_words.map(lambda l: list(map(enc_d.get, l)))

CPU times: user 4.53 s, sys: 52 ms, total: 4.58 s
Wall time: 4.58 s


In [68]:
data_words_enc.to_msgpack('../data/amazon/food/reviews_txt_enc_s.msg', compress='blosc')

In [69]:
pickle.dump(list(le.classes_), open('../data/amazon/food/vocab.p', 'wb'))